In [2]:
import os
import cv2
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

db_dir = os.path.join('.', 'tuberculosis-dataset')  # Relative path to dataset folder
normaldir = os.path.join(db_dir, 'Normal')
tbdir     = os.path.join(db_dir, 'Tuberculosis')

print("Normal directory exists?", os.path.exists(normaldir))
print("Tuberculosis directory exists?", os.path.exists(tbdir))


Normal directory exists? True
Tuberculosis directory exists? True


In [3]:
import cv2

images = []
labels = []
image_size = 256

# Load Normal images (label = 0)
for filename in os.listdir(normaldir):
    filepath = os.path.join(normaldir, filename)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (image_size, image_size))
        images.append(img)
        labels.append(0)

# Load Tuberculosis images (label = 1)
for filename in os.listdir(tbdir):
    filepath = os.path.join(tbdir, filename)
    img = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img = cv2.resize(img, (image_size, image_size))
        images.append(img)
        labels.append(1)

images = np.array(images)
labels = np.array(labels)
print("Images shape:", images.shape)
print("Labels shape:", labels.shape)


Images shape: (4200, 256, 256)
Labels shape: (4200,)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels, test_size=0.3, random_state=42
)

# Convert from [0,255] -> [0,1]
X_train = X_train.astype('float32') / 255.0
X_test  = X_test.astype('float32')  / 255.0

print("X_train shape:", X_train.shape)
print("X_test shape:",  X_test.shape)


X_train shape: (2940, 256, 256)
X_test shape: (1260, 256, 256)


In [5]:
smote = SMOTE(random_state=42)

# Flatten images for SMOTE
n_train    = X_train.shape[0]
X_train_2d = X_train.reshape(n_train, -1)  # shape: (n_samples, 256*256)

X_train_res, y_train_res = smote.fit_resample(X_train_2d, y_train)

# Reshape back to (H,W,1)
X_train_res = X_train_res.reshape(-1, image_size, image_size, 1)

print("After SMOTE, X_train_res shape:", X_train_res.shape)
print("After SMOTE, y_train_res shape:", y_train_res.shape)
unique_vals, counts = np.unique(y_train_res, return_counts=True)
print("Label distribution:", dict(zip(unique_vals, counts)))


After SMOTE, X_train_res shape: (4914, 256, 256, 1)
After SMOTE, y_train_res shape: (4914,)
Label distribution: {np.int64(0): np.int64(2457), np.int64(1): np.int64(2457)}


In [6]:
class TBChestXRayDataset(Dataset):
    def __init__(self, images, labels):
        """
        images: NumPy array of shape (N, H, W, 1)
        labels: NumPy array of shape (N,)
        """
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        # Convert (H,W,1) -> (1,H,W) for PyTorch (channels first)
        image = self.images[index].transpose(2,0,1)  # shape: (1,256,256)
        label = self.labels[index].astype('float32')
        
        # Turn into PyTorch tensors
        image_tensor = torch.tensor(image, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        
        return image_tensor, label_tensor

# Create datasets
train_dataset = TBChestXRayDataset(X_train_res, y_train_res)
# For test set, reshape to (N,H,W,1) as well
X_test_4d = X_test.reshape(-1, image_size, image_size, 1)
test_dataset  = TBChestXRayDataset(X_test_4d, y_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=32, shuffle=False)


In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        
        # We need to figure out the final dimension after these conv/pool layers
        # 256x256 -> after 3 conv+pool (each pool halves dimension):
        #   conv => dimension - 2
        #   pool => dimension / 2
        # Let's let PyTorch handle the shape in forward() or do a quick pass.
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # 64 channels * 30 * 30 = 57600 if dimension goes exactly as (30,30)
            nn.Linear(64 * 30 * 30, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

print(model)


SimpleCNN(
  (features): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=57600, out_features=64, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=64, out_features=1, bias=True)
    (5): Sigmoid()
  )
)


In [8]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# We want to reduce LR on plateau of "accuracy" which is a "maximize" metric
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.1, patience=1, min_lr=1e-5, verbose=True
)


f:\Conda\envs\py310\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [9]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    
    for imgs, labels in loader:
        imgs = imgs.to(device)
        labels = labels.to(device).view(-1, 1)  # (batch_size,1)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * imgs.size(0)
        
        preds = (outputs >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = total_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1, 1)
            
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item() * imgs.size(0)
            
            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    epoch_loss = total_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

num_epochs = 10
best_acc = 0.0
save_path = "tb_chest_xray_cnn_best.pt"

for epoch in range(num_epochs):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    
    # In many workflows, you'd do a separate validation set. Here, we'll 
    # just use training accuracy for the scheduler. You can adapt if you have val_loader.
    scheduler.step(train_acc)
    
    print(f"[Epoch {epoch+1}/{num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Save the best model based on training accuracy
    if train_acc > best_acc:
        best_acc = train_acc
        torch.save(model.state_dict(), save_path)
        print(f"Model improved at epoch {epoch+1}, saved to {save_path}")


[Epoch 1/10] Train Loss: 0.2405, Train Acc: 0.8948
Model improved at epoch 1, saved to tb_chest_xray_cnn_best.pt
[Epoch 2/10] Train Loss: 0.1277, Train Acc: 0.9512
Model improved at epoch 2, saved to tb_chest_xray_cnn_best.pt
[Epoch 3/10] Train Loss: 0.0761, Train Acc: 0.9733
Model improved at epoch 3, saved to tb_chest_xray_cnn_best.pt
[Epoch 4/10] Train Loss: 0.0491, Train Acc: 0.9827
Model improved at epoch 4, saved to tb_chest_xray_cnn_best.pt
[Epoch 5/10] Train Loss: 0.0788, Train Acc: 0.9723
[Epoch 6/10] Train Loss: 0.0288, Train Acc: 0.9886
Model improved at epoch 6, saved to tb_chest_xray_cnn_best.pt
[Epoch 7/10] Train Loss: 0.0241, Train Acc: 0.9919
Model improved at epoch 7, saved to tb_chest_xray_cnn_best.pt
[Epoch 8/10] Train Loss: 0.0291, Train Acc: 0.9896
[Epoch 9/10] Train Loss: 0.0346, Train Acc: 0.9882
[Epoch 10/10] Train Loss: 0.0202, Train Acc: 0.9945
Model improved at epoch 10, saved to tb_chest_xray_cnn_best.pt


In [10]:
# Load best model weights (if desired)
model.load_state_dict(torch.load(save_path))
model.eval()

all_preds = []
all_targets = []

with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device)
        
        outputs = model(imgs)
        preds = (outputs >= 0.5).float()
        
        all_preds.extend(preds.cpu().numpy().flatten())
        all_targets.extend(labels.cpu().numpy().flatten())

print("CLASSIFICATION REPORT:")
print(classification_report(all_targets, all_preds, digits=4))

print("CONFUSION MATRIX:")
print(confusion_matrix(all_targets, all_preds))


C:\Users\dell\AppData\Local\Temp\ipykernel_13340\2882599951.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path))


CLASSIFICATION REPORT:
              precision    recall  f1-score   support

         0.0     0.9923    0.9827    0.9875      1043
         1.0     0.9207    0.9631    0.9414       217

    accuracy                         0.9794      1260
   macro avg     0.9565    0.9729    0.9645      1260
weighted avg     0.9799    0.9794    0.9795      1260

CONFUSION MATRIX:
[[1025   18]
 [   8  209]]
